In [1]:
! ls -alh /var/genie-in-the-box/src/conf/long-term-memory/

total 40K
drwxr-xr-x 2 1000 1000 4.0K Aug  9 19:33 .
drwxr-xr-x 4 1000 1000 4.0K Jul 19 20:04 ..
-rw-r--r-- 1 1000 1000 4.0K Jul 19 20:04 ._events.jsonl
-rw-r--r-- 1 1000 1000  16K Aug  9 19:33 events.csv
-rw-r--r-- 1 1000 1000 9.9K Aug  9 19:33 events.jsonl


In [2]:
# ! pip install python-dotenv
! pip install langchain
! pip install langchain[docarray]
! pip install tiktoken
! pip install pinecone-client
! pip install jq
! pip install chromadb
! pip install openai[embeddings]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 8.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.3/215.3 kB 3.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.3/140.3 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.7/239.7 kB 38.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 13.5 MB/s eta 0:00:00
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.w

In [13]:
import os

import pandas as pd

from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [5]:
path = "/var/genie-in-the-box/src/lib"
sys.path.append( path )
import util as du
du.add_to_path( path )
# sys.path

Path [/var/genie-in-the-box/src/lib] already in sys.path


In [1]:
# import util as du
du.print_banner( "foo?" )

NameError: name 'du' is not defined

In [14]:
# Load CSV data From file into the Pandas DataFrame.
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   start_date                  200 non-null    object
 1   end_date                    200 non-null    object
 2   start_time                  200 non-null    object
 3   end_time                    200 non-null    object
 4   event_type                  200 non-null    object
 5   recurrent                   200 non-null    bool  
 6   recurrence_interval         47 non-null     object
 7   priority_level              200 non-null    object
 8   name                        200 non-null    object
 9   relationship                200 non-null    object
 10  description_who_what_where  200 non-null    object
dtypes: bool(1), object(10)
memory usage: 15.9+ KB


In [15]:
df.head()

,start_date,end_date,start_time,end_time,event_type,recurrent,recurrence_interval,priority_level,name,relationship,description_who_what_where
0,2023-07-01,2023-07-04,00:00,23:59,Concert,False,NaN,none,Jenny,coworker,Concert of Jenny at the city center
1,2023-07-01,2023-07-01,05:25,17:22,TODO,False,NaN,highest,Gregorio,friend,Send out invitations for the party for Gregori...
2,2023-07-01,2023-07-01,13:27,01:59,Appointment,False,NaN,high,Leroy Ruiz,father,Appointment with Leroy Ruiz at the clinic
3,2023-07-02,2023-07-03,13:18,02:40,Interview,False,NaN,highest,John,coworker,Job interview with John at Google
4,2023-07-03,2023-07-04,00:00,23:59,Performance,False,NaN,medium,Sue,sister,Theatre performance with Sue


### Count events by type

In [16]:
def get_event_count( df, event ):

    print( "Counting [{}] events...".format( event ), end="\n\n" )

    count = df[ df.event_type == event ].shape[ 0 ]

    print()
    print( "Done! Total {} {} events".format( count, event ) )

get_event_count( df, "Birthday" )

Counting [Birthday] events...


Done! Total 16 Birthday events


In [28]:
template_string = """
You are an event planner with proven capacity for querying pandas dataframes.
The dataframe that I am providing you for inspection contains two important fields:
1) event_type: tells us what kind of event is taking place, such as birthday, conference, meetings, etc.
2) description_who_what_where: tells us who, what and/or where the event is about, as in 'my friend's birthday at their favorite bar', the name of a conference, or what a meeting is about, e.g.: 'yet another boring meeting with the boss'.
I expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.
I expect you to remember query techniques and solutions that work, so that you do not have to begin from zero.
When you recognize a new query as similar to a previous query, say so
NEVER display raw dataframe content, unless specifically requested to do otherwise.
All dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.
When I ask you about events contained in this dataframe, you will always sort the events in ascending order, 1 per line, unless specifically requested to do otherwise.
When summarizing your results you will include the event type, start time, date (e.g.: today, tomorrow, Wednesday the 10th, etc.), and what the event is about, using conversational English, unless specifically requested to do otherwise.
DO NOT prepend numbers to a list of events, unless specifically requested to do otherwise.
If the question, delimited by three hash symbols ###, is vague or difficult to answer, please ask for clarification.
Given that, answer the following question: ###{question}###
"""

event_prompt_template = ChatPromptTemplate.from_template( template_string )
# print( prompt_template, end="\n\n" )

event_prompt = event_prompt_template.format_messages( question="How many records are in this dataset?" )
print( event_prompt, end="\n\n" )

# response_autoencoder = llm( prompt_autoencoder )
# print( response_autoencoder.content, end="\n\n" )

[HumanMessage(content="\nYou are an event planner with proven capacity for querying pandas dataframes.\nThe dataframe that I am providing you for inspection contains two important fields:\n1) event_type: tells us what kind of event is taking place, such as birthday, conference, meetings, etc.\n2) description_who_what_where: tells us who, what and/or where the event is about, as in 'my friend's birthday at their favorite bar', the name of a conference, or what a meeting is about, e.g.: 'yet another boring meeting with the boss'.\nI expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.\nI expect you to remember query techniques and solutions that work, so that you do not have to begin from zero.\nWhen you recognize a new query as similar to a previous query, say so\nNEVER display raw dataframe content, unless specifically requested to do otherwise.\nAll dates and temporal references like 'today', 'yesterday', 'next week' sho

In [29]:
llm      = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )
df_agent = create_pandas_dataframe_agent( llm, df, verbose=True )

In [1]:
import time

class Stopwatch:

    def __enter__( self ):

        self.start = time.time()

        return self

    def __exit__( self, *args ):

        self.end      = time.time()
        self.interval = int( ( self.end - self.start ) * 1000 )

        print( f'Elapsed time: {self.interval} milliseconds' )



In [54]:
def print_events( response ):

    events = response.split( "\n" )#[ 1: ]

    for event in events: print( "[{}]".format( event ) )

In [55]:
# Create function to take a question and call the data frame agent.
def query_dataframe( question ):

    with Stopwatch() as t:

        event_prompt = event_prompt_template.format_messages( question=question )
        response = df_agent.run( event_prompt )

    return response

response = query_dataframe( "What day is today? What day was yesterday? What day is tomorrow?" )
response



> Entering new AgentExecutor chain...
Thought: The question is asking for the current date, the date of yesterday, and the date of tomorrow. I can use Python's datetime library to get these dates.
Action: python_repl_ast
Action Input: 
```python
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()

# Get yesterday's date
yesterday = today - timedelta(days=1)

# Get tomorrow's date
tomorrow = today + timedelta(days=1)

today, yesterday, tomorrow
```

Observation: (datetime.datetime(2023, 8, 11, 17, 4, 31, 178022), datetime.datetime(2023, 8, 10, 17, 4, 31, 178022), datetime.datetime(2023, 8, 12, 17, 4, 31, 178022))
Thought:I now know the dates for today, yesterday, and tomorrow.
Final Answer: Today is August 11, 2023. Yesterday was August 10, 2023. Tomorrow will be August 12, 2023.

> Finished chain.
Elapsed time: 16195 milliseconds


'Today is August 11, 2023. Yesterday was August 10, 2023. Tomorrow will be August 12, 2023.'

In [21]:
response = query_dataframe( "How many records are in this dataset?" )
response



> Entering new AgentExecutor chain...
Thought: The question is asking for the total number of records in the dataframe. This can be achieved by using the `shape` attribute of the dataframe, which returns a tuple where the first element is the number of rows (records) and the second element is the number of columns.
Action: python_repl_ast
Action Input: df.shape[0]
Observation: 200
Thought:I now know the final answer
Final Answer: There are 200 records in this dataset.

> Finished chain.


'There are 200 records in this dataset.'

In [22]:
response = query_dataframe( "How many different types of events do I have?" )
response



> Entering new AgentExecutor chain...
Thought: To find out how many different types of events are in the dataframe, I need to look at the unique values in the 'event_type' column.
Action: python_repl_ast
Action Input: df['event_type'].nunique()
Observation: 12
Thought:I now know the final answer
Final Answer: There are 12 different types of events.

> Finished chain.


'There are 12 different types of events.'

In [23]:
response = query_dataframe( "How many TODO events do I have?" )
response



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe where the event_type is 'TODO'.
Action: python_repl_ast
Action Input: df[df['event_type'] == 'TODO'].shape[0]
Observation: 14
Thought:I now know the final answer
Final Answer: You have 14 TODO events.

> Finished chain.


'You have 14 TODO events.'

In [24]:
get_event_count( df, "TODO" )

Counting [TODO] events...


Done! Total 14 TODO events


In [20]:
response = query_dataframe( "Count and display my birthday events in August" )
response



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe to only include events that occur in August and have 'birthday' in their description. Then, I can count the number of these events.
Action: python_repl_ast
Action Input: df[(df['start_date'].str.contains('-08-')) & (df['description'].str.contains('birthday'))].shape[0]
Observation: AttributeError: Can only use .str accessor with string values!
Thought:The error suggests that the 'start_date' column is not of string type. I need to convert it to string before applying the str.contains() function.
Action: python_repl_ast
Action Input: df['start_date'] = df['start_date'].astype(str)
df[(df['start_date'].str.contains('-08-')) & (df['description'].str.contains('birthday'))].shape[0]
Observation: 11
Thought:I now know the final answer
Final Answer: There are 11 birthday events in August.

> Finished chain.


'There are 11 birthday events in August.'

In [25]:
response = query_dataframe( "Who has a birthday in August?" )
response



> Entering new AgentExecutor chain...
Thought: To find out who has a birthday in August, I need to filter the dataframe where the event_type is 'Birthday' and the start_date or end_date is in August. However, I don't see any 'Birthday' event in the given dataframe. I will first check if there are any 'Birthday' events in the dataframe.
Action: python_repl_ast
Action Input: df['event_type'].unique()
Observation: ['Concert' 'TODO' 'Appointment' 'Interview' 'Performance' 'Subscription'
 'Anniversary' 'Birthday' 'Workout' 'Workshop' 'Conference' 'Meeting']
Thought:There are 'Birthday' events in the dataframe. Now I will filter the dataframe for 'Birthday' events and check if any of them are in August.
Action: python_repl_ast
Action Input: df_birthday = df[df['event_type'] == 'Birthday']
df_birthday['start_date'] = pd.to_datetime(df_birthday['start_date'])
df_birthday_august = df_birthday[df_birthday['start_date'].dt.month == 8]
df_birthday_august['description_who_what_where']
Observation

<string>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


I now have a list of people who have a birthday in August. The list includes Barbara Jane Ruiz, Leroy Ruiz, Alice, Sue, Juan, and Gregorio. 
Final Answer: The people who have a birthday in August are Barbara Jane Ruiz, Leroy Ruiz, Alice, Sue, Juan, and Gregorio.

> Finished chain.


'The people who have a birthday in August are Barbara Jane Ruiz, Leroy Ruiz, Alice, Sue, Juan, and Gregorio.'

In [37]:
events = query_dataframe( "What events do I have this week?" )
# response
print_events( events )



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe to only include events that are happening this week. I will need to use Python's datetime library to determine the dates for this week. Then, I can filter the dataframe based on these dates.
Action: python_repl_ast
Action Input: import datetime
Observation: 
Thought:Now that I have imported the datetime library, I can use it to determine the dates for this week. This week starts on the Monday of the current week and ends on the Sunday of the current week. I will calculate these dates.
Action: python_repl_ast
Action Input: today = datetime.date.today()
weekday = today.weekday()
start_week = today - datetime.timedelta(days=weekday)
end_week = start_week + datetime.timedelta(days=6)
start_week, end_week
Observation: (datetime.date(2023, 8, 7), datetime.date(2023, 8, 13))
Thought:The start of this week is August 7, 2023 and the end of this week is August 13, 2023. Now I will filter the 

"Here are the events for this week:\n- Conference on Monday, August 07 at 00:00: Conference with Jenny on AI advancements\n- Concert on Monday, August 07 at 00:00: Concert of Inash at the city center\n- Workshop on Monday, August 07 at 14:43: Exciting workshop with Tom Ruiz on AGI\n- Birthday on Wednesday, August 09 at 00:00: Alice's birthday party at their favorite bar\n- Conference on Wednesday, August 09 at 00:00: Conference with Inash on AI advancements\n- Appointment on Thursday, August 10 at 16:22: Appointment with Barbara Jane Ruiz at the clinic\n- Interview on Thursday, August 10 at 22:02: Job interview with Gregorio at Google\n- Anniversary on Friday, August 11 at 00:00: John's anniversary celebration at the park\n- Birthday on Friday, August 11 at 00:00: Sue's birthday party at their favorite bar\n- Birthday on Friday, August 11 at 00:00: Leroy Ruiz's birthday party at their favorite bar\n- Appointment on Friday, August 11 at 00:03: Appointment with Jenny at the clinic\n- App

In [60]:
todays_events = query_dataframe( "What events do I have today?" )
print_events( todays_events )



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe to only include events that are happening today. I can do this by comparing the 'start_date' and 'end_date' columns to today's date. I will use the datetime library to get today's date.
Action: python_repl_ast
Action Input: import datetime
today = datetime.date.today()
today
Observation: 2023-08-11
Thought:Now that I have today's date, I can filter the dataframe to only include events that are happening today. This means that the 'start_date' is on or before today and the 'end_date' is on or after today.
Action: python_repl_ast
Action Input: df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])
today_events = df[(df['start_date'].dt.date <= today) & (df['end_date'].dt.date >= today)]
today_events
Observation:     start_date   end_date start_time end_time   event_type  recurrent  \
115 2023-08-07 2023-08-14      00:00    23:59   Conferenc

In [50]:
with Stopwatch() as t:

    response = query_dataframe( "What are your instructions?" )

response

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new AgentExecutor chain...
Thought: The instructions are clear and detailed. They outline the expectations for how I should interact with the provided dataframe, how I should format my responses, and how I should handle dates and times. The instructions also emphasize the importance of remembering successful query techniques and solutions, and recognizing when a new query is similar to a previous one. 
Final Answer: The instructions are to query the provided pandas dataframe, which contains information about various events. I should answer questions about the data in plain English, unless otherwise specified. I should remember successful query techniques and solutions, and recognize when a new query is similar to a previous one. I should not display raw dataframe content unless specifically asked to do so. I should handle all dates and temporal references using Python's datetime library. When asked about events, I should sort them in ascending order and provide one event p

"The instructions are to query the provided pandas dataframe, which contains information about various events. I should answer questions about the data in plain English, unless otherwise specified. I should remember successful query techniques and solutions, and recognize when a new query is similar to a previous one. I should not display raw dataframe content unless specifically asked to do so. I should handle all dates and temporal references using Python's datetime library. When asked about events, I should sort them in ascending order and provide one event per line, unless otherwise specified. I should summarize my results by including the event type, start time, date, and a description of the event, using conversational English, unless otherwise specified. I should not prepend numbers to a list of events, unless specifically asked to do so. If a question is vague or difficult to answer, I should ask for clarification."

In [ ]:
with Stopwatch() as t:

    response = query_dataframe( "What are your instructions?" )

response